In [52]:
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import timedelta,datetime
from openai import OpenAI
from langchain.messages import HumanMessage
import json

In [62]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get current time in HH:MM:SS format",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_date",
            "description": "Get current date in YYYY-MM-DD format",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "Get current date and time in YYYY-MM-DD HH:MM:SS format",
            "parameters": {"type": "object", "properties": {}},
        },
    },
]


# Functions
def get_current_time():
    return datetime.now().strftime("%H:%M:%S")


def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")


def get_current_datetime():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


function_map = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_current_datetime": get_current_datetime,
}

In [54]:
client=OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

messages=[]
messages.append(
    {
        "role":"user",
        "content":"What is the exact time formated as HH:MM:SS"
    }
)

completion = client.chat.completions.create(
    model="openrouter/free",
    messages=messages,
    tools=tools
)

completion

ChatCompletion(id='gen-1770049386-8OfE2AdwB5QKxSYJjvdY', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='chatcmpl-tool-804741a050d1648b', function=Function(arguments='{}', name='get_current_time'), type='function', index=0)], reasoning='User asks: "What is the exact time formatted as HH:MM:SS". We need to get current time. Use function get_current_time.', reasoning_details=[{'format': 'unknown', 'index': 0, 'type': 'reasoning.text', 'text': 'User asks: "What is the exact time formatted as HH:MM:SS". We need to get current time. Use function get_current_time.'}]), native_finish_reason='tool_calls')], created=1770049390, model='openai/gpt-oss-20b:free', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=51, prompt_tokens=183, total_tok

## Extracting the tool_call

In [55]:
tool_call = completion.choices[0].message.tool_calls[0]
print(f"AI wants to call: {tool_call.function.name}")
print(f"With arguments: {tool_call.function.arguments}")

AI wants to call: get_current_time
With arguments: {}


## Executing the function

In [ ]:
import json
args = json.loads(tool_call.function.arguments)

func_name=tool_call.function.name
func=function_map[func_name]
result=func(**args)

'22:01:57'

## Sending the function result back to the AI model

In [57]:
messages.append(completion.choices[0].message)
messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": result})

# Getting final answer
response2 = client.chat.completions.create(
    model="meta-llama/llama-3.3-70b-instruct:free",
    messages=messages,
    tools=tools
)

print("\n🤖 AI's final answer:")
print(response2.choices[0].message.content)


🤖 AI's final answer:
The current time is 21:53:11.
